In [1]:
# clear all the variables
%reset -f
import numpy as np
import pandas as pd
import sys
import turicreate as tc
import os
from os import walk
import scipy.io
import xlrd, os, fnmatch
import pandas as pd
import tensorflow as tf
import plotly.plotly as py
import plotly.graph_objs as go
from scipy.spatial.distance import cdist
# Path variable (where the data is located)
myPath = './data/RSVP_spatial_cue_extended_letter_duration/' 
# Append the path variable to existing search path
sys.path.append(myPath)
# Get the file information in the directory
file_list = []
ignore_list = ['pilot']
for root, dirs, files in os.walk(myPath):
    if 'pilot' in dirs:
        dirs[:]=[]
    for filename in files:
        if fnmatch.fnmatch(filename.lower(), "*csv"):
            file_list.append(os.path.join(root, filename))

/Users/supadhy6/miniconda3/envs/adiLab/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


In [2]:
file_list

['./data/RSVP_spatial_cue_extended_letter_duration/4_15Feb2019_11-23.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/8_16Feb2019_09-02.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/12_16Feb2019_11-04.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/14_16Feb2019_14-33.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/11_16Feb2019_10-36.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/5_15Feb2019_11-57.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/15_16Feb2019_15-03.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/16_16Feb2019_15-39.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/6_15Feb2019_12-29.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/10_16Feb2019_10-04.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/1_14Feb2019_14-19.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/17_16Feb2019_16-05.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/3_14Feb2019_15-17.csv',
 './data/RSVP_spat

In [3]:
data = tc.SFrame.read_csv(file_list[0])
temp = data
for i in range(1,len(file_list)):
    temp = tc.SFrame.read_csv(file_list[i])
    #temp.column_types([str,int,int,str,float,int,str,str,float,float,list,int,int])
    data = data.append(temp)
len(data)

Finished parsing file /Users/supadhy6/Documents/GitHub/spatio_temporal_information_processing/RSVP_visual_now/data/RSVP_spatial_cue_extended_letter_duration/4_15Feb2019_11-23.csv

Parsing completed. Parsed 100 lines in 0.03097 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,str,float,int,str,str,float,float,array,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/supadhy6/Documents/GitHub/spatio_temporal_information_processing/RSVP_visual_now/data/RSVP_spatial_cue_extended_letter_duration/4_15Feb2019_11-23.csv

Parsing completed. Parsed 280 lines in 0.008436 secs.

...

Parsing completed. Parsed 100 lines in 0.011364 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,str,float,int,str,str,float,float,array,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/supadhy6/Documents/GitHub/spatio_temporal_information_processing/RSVP_visual_now/data/RSVP_spatial_cue_extended_letter_duration/2_14Feb2019_14-45.csv

Parsing completed. Parsed 280 lines in 0.00912 secs.

4760

In [4]:
from plotly import tools
import colorlover as cl
trace = [[]]*2
ecc_length = len(data['cueEccentricity1'].unique())
colors = cl.scales['3']['seq']['YlOrRd']
for i,cuePos in enumerate(data['cueSpatialPos1'].unique()):
    cuePosdata = data[data['cueSpatialPos1']==cuePos]
    temp1 = []
    for j,ecc in enumerate(cuePosdata['cueEccentricity1'].unique().sort()):
        ecc_data = cuePosdata[cuePosdata['cueEccentricity1']==ecc]
        temp1.append(go.Histogram(x = ecc_data['responsePosRelative0'].to_numpy(), 
                                  histnorm='probability',name = str(ecc) + 'deg', opacity = 1, marker = dict(color = colors[j])))
    trace[i] = temp1

if data['cueSpatialPos1'].unique()[0] == [-1.0,0.0]:
    titles = ('West','East')
else:
    titles = ('East','West')
    
fig = tools.make_subplots(rows=1, cols=2, subplot_titles = titles)
fig['layout'].update(barmode='group', title = 'performance in RSVP visual now task for different cue positions - '+str(int(len(data)/280))+' participants')
#fig['layout']['xaxis3'].update(title='position of reported item relative to the true item', range = [-2,2])
#fig['layout']['xaxis4'].update(title='position of reported item relative to the true item', range = [-2,2])
fig['layout']['xaxis1'].update(range=[-3,3],title='position of reported item relative to the true item')
fig['layout']['xaxis2'].update(range=[-3,3],title='position of reported item relative to the true item')
fig['layout']['yaxis1'].update(title='Probability')
#fig['layout']['yaxis3'].update(title='Frequency')
for i in range(ecc_length):
    fig.add_trace(trace[0][ecc_length-1-i], 1, 1)
    fig.add_trace(trace[1][ecc_length-1-i], 1, 2)
    #fig.append_trace(trace[2][i], 2, 1)
    #fig.append_trace(trace[3][i], 2, 2)
py.iplot(fig, filename='RSVP_visual_now')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



/Users/supadhy6/miniconda3/envs/adiLab/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [5]:
trace_data = []
for j,ecc in enumerate(cuePosdata['cueEccentricity1'].unique().sort()):
        ecc_data = cuePosdata[cuePosdata['cueEccentricity1']==ecc]
        trace_data.append(go.Histogram(x = ecc_data['responsePosRelative0'].to_numpy(), 
                                  histnorm='probability',name = str(ecc) + 'deg', opacity = 1, marker = dict(color = colors[j])))

        
fig = tools.make_subplots(rows=1, cols=1)
fig['layout'].update(barmode='group', title = 'performance in RSVP visual now task for different cue positions - '+str(int(len(data)/280))+' participants', height=600,width = 650)
fig['layout']['xaxis1'].update(range = [-3,3], title='position of reported item relative to the true item')
fig['layout']['yaxis1'].update(title='Probability', range = [0,0.8])
for i in range(ecc_length):
    fig.add_trace(trace_data[ecc_length -1 - i], 1, 1)
py.iplot(fig, filename='RSVP_visual_now_grouped')

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [6]:
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde

In [7]:
X_plot = np.linspace(-3, 3, 1000)
trace_data = []
distributions = []
fill_colors = ['rgba(255,237,160,0.2)', 'rgba(254,178,76,0.2)', 'rgba(240,59,32,0.2)']
for j,ecc in enumerate(cuePosdata['cueEccentricity1'].unique().sort()):
        ecc_data = cuePosdata[cuePosdata['cueEccentricity1']==ecc]
        X = ecc_data['responsePosRelative0'].to_numpy()
        # Gaussian KDE
        #kde = KernelDensity(kernel='gaussian').fit(X)
        kde = gaussian_kde(X)
        log_dens = kde(X_plot)
        distributions.append({'ecc' : ecc, 'dist' : X, 'dens': log_dens})
        trace_data.append(go.Scatter(x = X_plot, y = log_dens,
                                  mode='lines', fill = 'tozeroy', fillcolor = fill_colors[j], name = str(ecc) + 'deg', line = dict(color = colors[j], width = 2) ))
        
fig = tools.make_subplots(rows=1, cols=1)
fig['layout'].update(barmode='group', title = 'performance in RSVP visual now task for different cue positions - '+str(int(len(data)/280))+' participants')
fig['layout']['xaxis1'].update(title='position of reported item relative to the true item')
fig['layout']['yaxis1'].update(title='Probability', range = [0,0.8])
for i in range(ecc_length):
    fig.add_trace(trace_data[i], 1, 1)
py.iplot(fig, filename='RSVP_visual_now_grouped')


This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [8]:
import numpy as np
from scipy.stats import kurtosis, skew

for x in distributions:
    print( 'Eccentricity: {}'.format( x['ecc']))
    print( 'excess kurtosis of normal distribution (should be 0): {}'.format( kurtosis(x['dens']) ))
    print( 'skewness of normal distribution (should be 0): {}'.format( skew(x['dens']) ))
    print( '\n')

Eccentricity: 2
excess kurtosis of normal distribution (should be 0): -1.4402792822399355
skewness of normal distribution (should be 0): 0.21824933444349992


Eccentricity: 10
excess kurtosis of normal distribution (should be 0): -1.4456160977696304
skewness of normal distribution (should be 0): 0.12346173688522705




In [9]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

hsb = data.to_dataframe()
mod = ols('responsePosRelative0 ~ C(cueEccentricity1)',
                data=hsb).fit()
                
aov_table = sm.stats.anova_lm(mod, typ=2)
print(aov_table)

                           sum_sq      df         F    PR(>F)
C(cueEccentricity1)     78.428782     1.0  7.011732  0.008124
Residual             53219.965966  4758.0       NaN       NaN


In [10]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[1]['ecc']))
scipy.stats.ttest_ind(distributions[0]['dist'], distributions[1]['dist'],axis =0, equal_var=True)

Ecc:2 vs.Ecc:10


Ttest_indResult(statistic=2.226825085604193, pvalue=0.026052279149974236)

In [11]:
# print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
# scipy.stats.ttest_ind(distributions[0]['dist'], distributions[2]['dist'], axis =0,equal_var=True)

In [12]:
# print('Ecc:{}'.format(distributions[1]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
# scipy.stats.ttest_ind(distributions[1]['dist'], distributions[2]['dist'], axis =0,equal_var=True)

In [111]:
import turicreate.aggregate as agg
sub_data = data.groupby(['subject','cueEccentricity1'],operations = {'avg_lag':agg.AVG('responsePosRelative0')})
ratios = []
c_val = [-1,0]
for sub in data['subject'].unique().sort():
    temp1 = data[data['subject']==sub]
    for ecc in temp1['cueEccentricity1'].unique().sort():
        temp = temp1[temp1['cueEccentricity1']==ecc]
        count_data = temp['responsePosRelative0'].value_counts()
        a = count_data[count_data['value']==c_val[0]]['count'].to_numpy()
        b = count_data[count_data['value']==c_val[1]]['count'].to_numpy()
        if len(a) > 0 and len(b)>0:
            ratios.extend(a-b)
        elif len(a)==0 and len(b)>0:
            ratios.extend(0-b)
        elif len(a)>0 and len(b)==0:
            ratios.extend(a-0)
        else:
            ratios.extend(0)
            
sub_data = sub_data.sort(['subject','cueEccentricity1'])
skewness = []
kurtosissy = []
for sub in data['subject'].unique().sort():
    subd = data[data['subject']==sub]
    for ecc in subd['cueEccentricity1'].unique().sort():
        eccd = subd[subd['cueEccentricity1']==ecc]
        X = eccd['responsePosRelative0'].to_numpy()
        kde = gaussian_kde(X)
        log_dens = kde(X_plot)
        skewness.append(skew(log_dens))
        kurtosissy.append(kurtosis(log_dens))

sub_data['skew'] = skewness
sub_data['kurtosis'] = kurtosissy
sub_data['ratio'] = ratios

In [112]:
sub_hsb = sub_data.to_dataframe()
#sub_hsb = hsb.groupby(['subject','cueEccentricity1']).apply(skew('responseRelative0'))
mod = ols('skew ~ C(cueEccentricity1)',
                data=sub_hsb).fit()
                
aov_table = sm.stats.anova_lm(mod)
aov_table

,df,sum_sq,mean_sq,F,PR(>F)
C(cueEccentricity1),1.0,0.099960,0.099960,0.472392,0.49684
Residual,32.0,6.771332,0.211604,NaN,NaN


In [113]:
mod1 = ols('kurtosis ~ C(cueEccentricity1)',
                data=sub_hsb).fit()
                
aov_table1 = sm.stats.anova_lm(mod1)
aov_table1

,df,sum_sq,mean_sq,F,PR(>F)
C(cueEccentricity1),1.0,0.208919,0.208919,0.673302,0.417974
Residual,32.0,9.929310,0.310291,NaN,NaN


In [114]:
deg2_data = sub_data[sub_data['cueEccentricity1']==2]
deg2_data = deg2_data.sort('subject')
#deg6_data = sub_data[sub_data['cueEccentricity1']==6]
#deg6_data = deg6_data.sort('subject')
deg10_data = sub_data[sub_data['cueEccentricity1']==10]
deg10_data = deg10_data.sort('subject')

deg2 = deg2_data['avg_lag'].to_numpy()
deg2_ratio = deg2_data['ratio'].to_numpy()
#deg6 = deg6_data['avg_lag'].to_numpy()
#deg6_ratio = deg6_data['ratio'].to_numpy()
deg10 = deg10_data['avg_lag'].to_numpy()
deg10_ratio = deg10_data['ratio'].to_numpy()

In [115]:
len(deg2_ratio)

17

In [116]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs. zero mean')
scipy.stats.ttest_1samp(deg2,0.0)

Ecc:2 vs. zero mean


Ttest_1sampResult(statistic=-2.848870559296973, pvalue=0.011608075116130462)

In [117]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[1]['ecc']))
scipy.stats.ttest_rel(deg2, deg10)

Ecc:2 vs.Ecc:10


Ttest_relResult(statistic=2.5999382095364325, pvalue=0.019343236325712963)

In [118]:
# print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
# scipy.stats.ttest_rel(deg2,deg10)

In [119]:
# print('Ecc:{}'.format(distributions[1]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
# scipy.stats.ttest_rel(deg6,deg10)

In [120]:
output = tc.SFrame()
output['subject'] = np.arange(1,len(deg2_ratio)+1)
output['avg_lag_2deg'] = deg2
#output['avg_lag_6deg'] = deg6
output['avg_lag_10deg'] = deg10

output['ratio_2deg'] = deg2_ratio
#output['ratio_6deg'] = deg6_ratio
output['ratio_10deg'] = deg10_ratio

In [121]:
output

subject,avg_lag_2deg,avg_lag_10deg,ratio_2deg,ratio_10deg
1,-0.1928571428571429,-0.392857142857143,-17,13
2,0.10714285714285707,-0.7785714285714284,3,18
3,0.0428571428571428,-0.442857142857143,-27,-15
4,-0.3285714285714285,-0.6214285714285716,-28,-16
5,-1.1785714285714286,-0.9142857142857141,35,67
6,-0.18571428571428567,-0.7142857142857141,-40,-13
7,-0.05714285714285714,-0.028571428571428602,-51,-19
8,-0.21428571428571422,-0.1357142857142858,-53,-46
9,-0.31428571428571433,-0.042857142857142885,-33,-24
10,-0.2571428571428571,-0.8285714285714287,-10,36


In [122]:
# print('Ecc:{}'.format(2)+' vs.'+ 'Ecc:{}'.format(6))
# scipy.stats.ttest_rel(output['avg_lag_2deg'].to_numpy(), output['avg_lag_6deg'].to_numpy())

In [123]:
print('Ecc:{}'.format(2)+' vs.'+ 'Ecc:{}'.format(10))
scipy.stats.ttest_rel(output['avg_lag_2deg'].to_numpy(), output['avg_lag_10deg'].to_numpy())

Ecc:2 vs.Ecc:10


Ttest_relResult(statistic=2.5999382095364325, pvalue=0.019343236325712963)

In [124]:
# print('Ecc:{}'.format(6)+' vs.'+ 'Ecc:{}'.format(10))
# scipy.stats.ttest_rel(output['avg_lag_6deg'].to_numpy(), output['avg_lag_10deg'].to_numpy())

In [125]:
# print('Ecc:{}'.format(2)+' vs.'+ 'Ecc:{}'.format(6))
# scipy.stats.ttest_rel(output['ratio_2deg'].to_numpy(), output['ratio_6deg'].to_numpy())

In [126]:
print('Ecc:{}'.format(2)+' vs.'+ 'Ecc:{}'.format(10))
scipy.stats.ttest_rel(output['ratio_2deg'].to_numpy(), output['ratio_10deg'].to_numpy())

Ecc:2 vs.Ecc:10


Ttest_relResult(statistic=-6.879173731240084, pvalue=3.7065193630143115e-06)

In [127]:
# print('Ecc:{}'.format(6)+' vs.'+ 'Ecc:{}'.format(10))
# scipy.stats.ttest_rel(output['ratio_6deg'].to_numpy(), output['ratio_10deg'].to_numpy())

In [129]:
trace_data = []
trace_data.append(go.Bar(x = ['preference to lag -1 over lag 1 responses'],
                        y = [output['ratio_2deg'].mean()],
                        name='2 deg',
                        marker = dict(color = colors[0]),
                        error_y = dict(type = 'data',
                                      array = [output['ratio_2deg'].std()/np.sqrt(14)],
                                      visible=True
                                      )
                        )
                 )

# trace_data.append(go.Bar(x = ['preference to lag -1 over lag 1 responses'],
#                         y = [output['ratio_6deg'].mean()],
#                         name='6 deg',
#                         marker = dict(color = colors[1]),
#                         error_y = dict(type = 'data',
#                                       array = [output['ratio_6deg'].std()/np.sqrt(14)],
#                                       visible=True
#                                       )
#                         )
#                  )

trace_data.append(go.Bar(x = ['preference to lag -1 over lag 1 responses'],
                        y = [output['ratio_10deg'].mean()],
                        name='10 deg',
                        marker = dict(color = colors[2]),
                        error_y = dict(type = 'data',
                                      array = [output['ratio_10deg'].std()/np.sqrt(14)],
                                      visible=True
                                      )
                        )
                 )

layout = go.Layout(
    barmode='group',
    title = str(int(len(data)/280))+' participants',
    xaxis = dict(zeroline=False),
    yaxis = dict(title = 'avg. # of responses lag('+str(c_val[0])+') - lag('+str(c_val[1])+')',zeroline=False),
)
fig = go.Figure(data=trace_data, layout=layout)
py.iplot(fig, filename='error-bar-bar')

/Users/supadhy6/miniconda3/envs/adiLab/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

